# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "41a27c87ecddd36fe4e892d04ee9ff488a9e48242f6026bfbb605abff370ef13"

In [2]:
from langchain_openai import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [3]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [10]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [11]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No. 
Final answer: 第28届大运会 (The 28th Universiade)

In [12]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output:  No.

In [13]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))

D:\miniconda3\envs\torch2\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




成都举办的大运会是第31届大运会。


In [14]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))



2023年大运会的举办地将在中国的重庆市。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [15]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [16]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [17]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the English translation of "2023年大运会"?
Intermediate answer: ['... Translation API 翻译。Switch to English 开发Gmail 解决方案。增强Gmail 体验借助 ... 2023年大运会推迟原因. 694 阅读， 11-16. 云南省运动会2023金牌榜. 694 阅读， 11 ...']


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse output: The provided intermediate answer does not give a clear translation of "2023年大运会". Let's clarify the translation:

The term "2023年大运会" translates to "2023 Universiade" in English.

Now, let's find the location of the 2023 Universiade:

Follow up: Where is the 2023 Universiade being held?
Final answer: The 2023 Universiade is scheduled to be held in Lake Placid, New York, United States.